<a href="https://colab.research.google.com/github/komo135/tensorflow-rl/blob/master/sac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

In [1]:
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [16]:
%cd /content/drive/My Drive

try:
  import imp
  imp.reload(sac)
except:
  import sac

agent = sac.Agent(spread=10, pip_cost=1000, leverage=400, min_lots=0.01, assets=10000, available_assets_rate=0.4,
                 restore=not True, step_size=int(96), n=3, lr=1e-5)

/content/drive/My Drive


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
%cd /content

agent.run()

action probability: buy=0.5104166666666666, sell=0.4895833333333333, hold=0.0
epoch: 5, total assets:8221.000000003358, growth_rate:0.8221000000003358

action probability: buy=0.40625, sell=0.59375, hold=0.0
epoch: 10, total assets:2031.0000000036418, growth_rate:0.20310000000036418

action probability: buy=0.4583333333333333, sell=0.5416666666666666, hold=0.0
epoch: 15, total assets:40494.00000002548, growth_rate:4.049400000002548

action probability: buy=0.53125, sell=0.46875, hold=0.0
epoch: 20, total assets:942.0000000015416, growth_rate:0.09420000000015416

action probability: buy=0.4583333333333333, sell=0.5416666666666666, hold=0.0
epoch: 25, total assets:11832.000000008433, growth_rate:1.1832000000008434

action probability: buy=0.5520833333333334, sell=0.4479166666666667, hold=0.0
epoch: 30, total assets:4186.000000007338, growth_rate:0.4186000000007338

action probability: buy=0.3854166666666667, sell=0.6145833333333334, hold=0.0
epoch: 35, total assets:516.0000000022053, gro

/content/drive/My Drive/new_rewards.py:199: RuntimeWarning: invalid value encountered in log
  self.growth_rate.append(np.log(undetermined_assets / self.initial_assets) * 100)


action probability: buy=0.9166666666666666, sell=0.08333333333333333, hold=0.0
epoch: 110, total assets:8751.000000001597, growth_rate:0.8751000000001598

action probability: buy=0.7395833333333334, sell=0.2604166666666667, hold=0.0
epoch: 115, total assets:2815.9999999998317, growth_rate:0.2815999999999832

action probability: buy=0.9375, sell=0.0625, hold=0.0
epoch: 120, total assets:14337.000000000558, growth_rate:1.433700000000056

action probability: buy=0.125, sell=0.875, hold=0.0
epoch: 125, total assets:26440.99999999821, growth_rate:2.644099999999821

action probability: buy=0.4583333333333333, sell=0.5416666666666666, hold=0.0
epoch: 130, total assets:-2238.5302734375, growth_rate:-0.22385302186012268

action probability: buy=0.78125, sell=0.21875, hold=0.0
epoch: 135, total assets:8937.999999999534, growth_rate:0.8937999999999534

action probability: buy=0.9375, sell=0.0625, hold=0.0
epoch: 140, total assets:-3419.140625, growth_rate:-0.3419140577316284

action probability: 

/content/drive/My Drive/new_rewards.py:199: RuntimeWarning: invalid value encountered in log
  self.growth_rate.append(np.log(undetermined_assets / self.initial_assets) * 100)


action probability: buy=0.875, sell=0.125, hold=0.0
epoch: 960, total assets:17758.00000000072, growth_rate:1.775800000000072

action probability: buy=0.07291666666666667, sell=0.9270833333333334, hold=0.0
epoch: 965, total assets:30044.73828125, growth_rate:3.004473924636841

action probability: buy=1.0, sell=0.0, hold=0.0
epoch: 970, total assets:19243.000000000167, growth_rate:1.9243000000000168

action probability: buy=0.6354166666666666, sell=0.3645833333333333, hold=0.0
epoch: 975, total assets:787.9999999996119, growth_rate:0.07879999999996119

action probability: buy=0.9583333333333334, sell=0.041666666666666664, hold=0.0
epoch: 980, total assets:12505.000000001273, growth_rate:1.2505000000001274

action probability: buy=0.5520833333333334, sell=0.4479166666666667, hold=0.0
epoch: 985, total assets:1409.999999998859, growth_rate:0.1409999999998859

action probability: buy=0.78125, sell=0.21875, hold=0.0
epoch: 990, total assets:4747.000000001948, growth_rate:0.4747000000001948


In [0]:
import numpy as np

for _ in range(1):
  # agent.test(spread=10, pip_cost=1000, los_cut=150,test_data= not True)
  tree_idx, replay = agent.memory.sample(128)

  states = np.array([a[0][0] for a in replay], np.float32)
  new_states = np.array([a[0][3] for a in replay], np.float32)
  actions = np.array([a[0][1] for a in replay]).reshape((-1, 4))
  rewards = np.array([a[0][2] for a in replay], np.float32).reshape((-1, 1))

  ent_coef = tf.exp(agent.model.log_ent_coef)
  ################################################################################
  with tf.GradientTape() as p_tape:
      d, policy, logp_pi = agent.model.actor.predict_on_batch(states)
      q1_pi, q2_pi, _ = agent.model.critic.predict_on_batch([states, policy])
      # q1, q2, v = self.model.critic.predict_on_batch([states, actions])
      min_q_pi = tf.minimum(q1_pi, q2_pi)
      # p_loss = -tf.reduce_mean(q1_pi) - tf.reduce_mean(q2_pi)
      # p_loss = tf.reduce_mean(ent_coef * logp_pi * 100 - q1_pi)
      p_loss = tf.reduce_mean(ent_coef * logp_pi * 2 - (q1_pi + q2_pi))
      # p_loss = tf.reduce_mean(ent_coef * logp_pi - q2_pi)
  ################################################################################
  with tf.GradientTape() as v_tape:
      _, _, target_v = agent.target_model.critic.predict_on_batch([new_states, actions])
      q1, q2, v = agent.model.critic.predict_on_batch([states, actions])
      min_q_pi = tf.minimum(q1_pi, q2_pi)

      q_backup = rewards + agent.gamma * target_v

      q1_loss = 0.5 * tf.reduce_mean((q_backup - q1) ** 2)
      q2_loss = 0.5 * tf.reduce_mean((q_backup - q2) ** 2)

      v_backup = min_q_pi - ent_coef * logp_pi
      v_loss = 0.5 * tf.reduce_mean((v_backup - v) ** 2)

      v_loss += q1_loss + q2_loss
  ################################################################################
  with tf.GradientTape() as e_tape:
      e_loss = -tf.reduce_mean(agent.model.log_ent_coef * logp_pi + agent.model.target_entropy)
  ################################################################################

  ae = np.abs(np.array(q_backup) - min_q_pi.numpy()).reshape((-1,))
  # print(ae)
  # agent.memory.batch_update(tree_idx, ae)

  gradients = v_tape.gradient(v_loss, agent.model.critic.trainable_variables)
  # gradients = [(tf.clip_by_value(grad, -1.0, 1.0))
  #              for grad in gradients]
  # agent.v_opt.apply_gradients(zip(gradients, agent.model.critic.trainable_variables))

  p_gradients = p_tape.gradient(p_loss, agent.model.actor.trainable_variables)
  # gradients = [(tf.clip_by_value(grad, -1.0, 1.0))
  #               for grad in gradients]

  # agent.p_opt.apply_gradients(zip(gradients, agent.model.actor.trainable_variables))

  # gradients = e_tape.gradient(e_loss, agent.model.log_ent_coef)
  # gradients = (tf.clip_by_value(gradients, -1.0, 1.0))
  # agent.e_opt.apply_gradients([[gradients, agent.model.log_ent_coef]])

In [9]:
i = 5
print(q2_pi[i])
print(q2[i])
print(v[i])
print(rewards[i])

tf.Tensor([-106.839226], shape=(1,), dtype=float32)
tf.Tensor([-106.41598], shape=(1,), dtype=float32)
tf.Tensor([-70.77481], shape=(1,), dtype=float32)
[891.8]


In [10]:
np.array([int((i1 - i2) * 10 ** 2) / (10 ** 2) for i1, i2 in zip(q2,q2_pi)]).reshape((-1,)).astype(np.float64)

array([-0.33,  0.04,  0.17, -0.11,  0.06,  0.42, -0.24,  0.03, -0.04,
        0.17,  0.03,  0.26, -0.4 , -0.12, -0.08,  0.29, -0.45, -0.21,
        0.12,  0.11,  0.31,  0.74, -0.03,  0.  , -0.22,  0.34,  0.02,
        0.44,  0.12, -0.17, -0.05, -0.38,  0.47, -0.06, -0.31,  0.11,
        0.24,  0.44,  0.67,  0.18, -0.33, -0.39, -0.08,  0.11,  0.23,
       -0.21, -0.2 , -0.02, -0.06,  0.32,  0.18,  0.44,  0.68, -0.45,
        0.11, -0.1 ,  0.09, -0.29,  0.24,  0.  ,  0.03, -0.12, -0.27,
       -0.22, -0.36, -0.05, -0.06, -0.13, -0.06, -0.26, -0.58,  0.34,
        0.04,  0.02,  0.41,  0.  ,  0.1 , -0.38,  0.52, -0.22, -0.07,
        0.13,  0.05, -0.39, -0.43,  0.32, -0.67, -0.16,  0.3 ,  0.03,
       -0.1 ,  0.42,  0.  , -0.02, -0.02,  0.08,  1.33, -0.51,  0.23,
        0.3 , -0.19, -0.52, -0.13, -0.25,  0.34])

In [15]:
rewards

array([13035.51], dtype=float32)

In [0]:
agent.policy_init()
agent.rewards.total_gain = []

In [7]:
agent.rewards.total_gain = []
agent.run(train=False)

action probability: buy=0.4583333333333333, sell=0.5416666666666666, hold=0.0
epoch: 6, total assets:28537.02420263714, growth_rate:0.2853702420263714

